### Imports

In [1]:
from Preprocessing import * 


### 1- Read Data

In [2]:
#1-Read Datasets
training_dataset = read_training_dataset()
dev_dataset = read_dev_dataset()

Read training set successfully
Read validation set successfully


### 2- Clean Data 

In [3]:
# 2-Data cleaning
# Define a regular expression pattern
# This pattern keeps Arabic letters, diacritics, and whitespaces and endlines
# pattern = re.compile(r"[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s\n]")
pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s,.؟،;:!?\-\'"]')



# Replace unmatched characters with an empty string

cleaned_corpus = [re.sub(pattern, "", t) for t in training_dataset]
cleaned_corpus = [re.sub("\s\s+", " ", c) for c in cleaned_corpus]
# re.sub("\s\s+" , " ", s)


print(len(cleaned_corpus))
# write the clean corpus to file
with open("cleaned_corpus.txt", "w", encoding="utf-8") as f:
    for l in cleaned_corpus:
        f.write(l + "\n")

50000


In [4]:
# data_with_labels = []
data = []
labels = []
# Example usage
for c in cleaned_corpus:
    sentences = re.split(r"[,.؟،;:!?']+", c)  # split on all punctuation
    labels += sentences

    without_dialects = [
        extract_arabic_letters(s) for s in sentences
    ]  # get the letters without dialects
    data += without_dialects


# remove any spaces from line
data = [d.strip() for d in data]
labels = [l.strip() for l in labels]

# remove empty lines
data = [i for i in data if i]
labels = [i for i in labels if i]
# data = list(filter(lambda x: x != "", data))
labels = list(filter(lambda x: x != "", labels))

# write the clean corpus to file
with open("training_data.txt", "w", encoding="utf-8") as f:
    for d in data:
        f.write(str(d) + "\n")

with open("training_labels.txt", "w", encoding="utf-8") as f:
    for l in labels:
        f.write(str(l) + "\n")

In [5]:
print(len(data))
print(len(labels))
# # data.remove("")
# print(data[9])


172228
172299


### 3- Tokenize Data

In [8]:

#3.1 Tokenize & stem words to vocab
vocab = set()
tokenized_data=[]
vocab, tokenized_data = tokenize_to_vocab(data, vocab)
stemmedVocab = []

for v in vocab:
    stemmedVocab.append(stem(v))
# stemmedVocab = set(stemmedVocab)

with open("vocab.txt", "w", encoding="utf-8") as f:
    for v in vocab:
        f.write(str(v) + "\n")

with open("stemmedvocab.txt", "w", encoding="utf-8") as f:
    for sv in stemmedVocab:
        f.write(str(sv) + "\n")

with open("tokenized_data_sentences.txt", "w", encoding="utf-8") as f:
    for ts in tokenized_data:
        f.write(str(ts) + "\n")


In [13]:
#3.1 Tokenize & stem words to vocab
diacritics = set()
tokenized_labels=[]
diacritics , tokenized_labels = tokenize_to_vocab(labels, vocab)


with open("tokenized_label_sentences.txt", "w", encoding="utf-8") as f:
    for tl in tokenized_labels:
        f.write(str(tl) + "\n")

with open("diacritics.txt", "w", encoding="utf-8") as f:
    for d in diacritics:
        f.write(str(d) + "\n")

with open("diacritics_encoded.txt", "w", encoding="utf-8") as f:
    for d in diacritics:
        f.write(str(extract_diacritics(str(d))) + "\n")

In [15]:
#Word Embeddings
pip install fasttext
import fasttext.util
fasttext.util.download_model('arabic.txt', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

ModuleNotFoundError: No module named 'fasttext'

In [14]:

# letters = u"هاربا"
# encoded_marks = 40610
# x= join_word_diacritics(letters, encoded_marks, "decimal")
# print(x)
# print(extract_diacritics(x))

هَارِبًا
40610
